<a href="https://colab.research.google.com/github/anujR1994/Data-Science-Projects/blob/main/Face_Mask_Detection_Model_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Mask Detection Model Using VGG16

This notebook outlines the steps to build a face mask detection model using the VGG16 architecture, as specified in the 'Mask Identification - Case Study'.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup Paths

In [3]:
train_dir = '/content/drive/MyDrive/GrayScale28X28MaskDataSet/train'
test_dir = '/content/drive/MyDrive/GrayScale28X28MaskDataSet/test'

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Step 2: Data Preprocessing

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

Found 1325 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


## Step 3: Model Building

In [6]:
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification (mask / no mask)

model = Model(base_model.input, x)

58889256/58889256 [==============================] - 0s 0us/step


## Step 4: Model Compilation

In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Step 5: Model Training

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=30,  # depends on your data
    epochs=5,
    validation_data=test_generator,
    validation_steps=50  # depends on your test data
)

Epoch 1/5
30/30 [==============================] - ETA: 0s - loss: 0.0922 - accuracy: 0.9726

30/30 [==============================] - 18s 606ms/step - loss: 0.0922 - accuracy: 0.9726 - val_loss: 0.0809 - val_accuracy: 0.9794
Epoch 2/5
30/30 [==============================] - 14s 453ms/step - loss: 0.0882 - accuracy: 0.9709
Epoch 3/5
30/30 [==============================] - 14s 466ms/step - loss: 0.0968 - accuracy: 0.9733
Epoch 4/5
30/30 [==============================] - 12s 405ms/step - loss: 0.1488 - accuracy: 0.9624
Epoch 5/5
30/30 [==============================] - 12s 376ms/step - loss: 0.2904 - accuracy: 0.9299


## Step 6: Evaluation

In [10]:
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Test accuracy:', test_acc)

10/50 [=====>........................] - ETA: 3s - loss: 0.0531 - accuracy: 0.9897

50/50 [==============================] - 1s 18ms/step - loss: 0.0531 - accuracy: 0.9897
Test accuracy: 0.9896907210350037
